# First Draft Of Credit Model - The Dummy Model (Auto Rejection)

## Parameters
- Company age
- Average growth 
- Whether or not they have a current delinquency
- Expected loss

In [703]:
import pandas as pd
import numpy as np
import json
json_file_path = "/Users/alizia/AliDivi/proof-of-concept/datasets/accounts.json"
df = pd.read_json(json_file_path)
# Load the JSON data
with open('/Users/alizia/AliDivi/proof-of-concept/datasets/accounts.json') as json_file:
    data = json.load(json_file)
# Normalize the nested JSON column
arr = data['data']

df = pd.DataFrame.from_dict(arr['results'])
df

,nickName,tradeName,companyName,stateRegistration,type,source,institutionType,businessType,socioEnvironmentalRisk,sourceDetails,...,foundationDate,contactData,bankData,document,activityCodes,addresses,shareholdingComposition,socinalAccountId,incompleteFields,id
0,Well Teste,None,None,None,0,0,2.0,2.0,0.0,None,...,2017-02-07T00:00:00+00:00,None,None,"{'number': '06996051000100', 'issuer': None, '...",[8800600],[],[],None,"[account.companyName, account.tradeName, accou...",c484d2e3-1c34-4f92-9044-8d232f3a313a
1,VP Produtos Digitais,VP Produtos Digitais,VP Produtos Digitais LTDA,3212315,0,0,0.0,2.0,1.0,None,...,2020-01-01T00:00:00+00:00,"{'email': 'vitor@digitalrake.tech', 'phone': '...","{'bankCode': '341', 'branchCode': '6664', 'acc...","{'number': '38015705000155', 'issuer': None, '...","[6203100, 6202300, 6201501]","[{'zipCode': '09230580', 'street': 'RUA SANTA ...","[{'signForTheCompany': True, 'phone': '1197459...",201347796,[],cc17ad4d-261f-48e9-93f0-dad39e733360
2,MCo,None,None,None,0,0,NaN,NaN,NaN,None,...,None,None,None,"{'number': '29233193000102', 'issuer': None, '...",[],[],[],None,"[account.companyName, account.tradeName, accou...",4cadadad-c9eb-4f9d-a436-e998a9c949fc
3,MCo,None,None,None,0,0,NaN,NaN,NaN,None,...,None,None,None,"{'number': '33445489000164', 'issuer': None, '...",[],[],[],None,"[account.companyName, account.tradeName, accou...",0cc2d041-040a-45da-866d-a837f8e1e3d7
4,MCo,None,None,None,0,0,NaN,NaN,NaN,None,...,None,None,None,"{'number': '43426608000195', 'issuer': None, '...",[],[],[],None,"[account.companyName, account.tradeName, accou...",13749595-f962-42d3-844d-a146c53e205a
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,MINI DELICIAS LOJA 2,MINI DELICIAS LOJA 2,ALAN NASCIMENTO CRUZ 05937497589,None,1,0,2.0,2.0,NaN,None,...,2020-06-16T00:00:00+00:00,"{'email': None, 'phone': None, 'website': '', ...","{'bankCode': None, 'branchCode': None, 'accoun...","{'number': '37424253000100', 'issuer': None, '...",[0111301],"[{'zipCode': '49020210', 'street': 'AV PRF ACR...",[],,"[account.stateRegistration, account.legalOffic...",fb6e1c16-7287-49bd-a82d-eb454da253d6
996,Kid's Brasil,Kid's Brasil,BRASIL IMPORTS EIRELI ME,52.162.733.402,2,0,2.0,1.0,0.0,None,...,2016-08-25T00:00:00+00:00,"{'email': 'financeiro@kidsbrasil.com.br', 'pho...","{'bankCode': '001', 'branchCode': '3689-7', 'a...","{'number': '26035247000137', 'issuer': None, '...",[4781400],"[{'zipCode': '74335100', 'street': 'R MACHADO ...","[{'signForTheCompany': True, 'phone': None, 'a...",200650603,[account.legalOfficer],3b4f1e6e-a2b3-4375-9e6f-2c5944faeea7
997,CONSULTORIA HARMONIA,CONSULTORIA HARMONIA,CONCEICAO A. DE OLIVEIRA,isento,2,0,2.0,2.0,0.0,None,...,2020-01-30T00:00:00+00:00,"{'email': 'amadeusvictor@hotmail.com', 'phone'...","{'bankCode': '033', 'branchCode': '44', 'accou...","{'number': '36187924000196', 'issuer': None, '...",[6619399],"[{'zipCode': '14811112', 'street': 'Avenida Es...","[{'signForTheCompany': True, 'phone': None, 'a...",202010021,[account.legalOfficer],b1749a0f-5e56-4430-ab48-42cee10095eb
998,TO BE CURSOS ONLINE,TO BE CURSOS ONLINE,HUGO SAMIR BECHARA ARRERO LIMA 99436400225,isento,2,0,2.0,2.0,0.0,None,...,2020-07-17T00:00:00+00:00,"{'email': 'samirarero@gmail.com', 'phone': '91...","{'bankCode': '341', 'branchCode': '1675', 'acc...","{'number': '37768555000198', 'issuer': None, '...",[8599604],"[{'zipCode': '66093605', 'street': 'AV ROMULO ...","[{'signForTheCompany': True, 'phone': None, 'a...",200627605,[account.legalOfficer],51ba4363-39b0-428f-8249-db79f27bd4b8


First, we have to see if the company age is less that 6 months old. This requires converting the foundationDate to DateTime objects. So let's simplify this table down to just the nickName and foundationDate first and then build on the data needed for the next set of auto rejections.

In [704]:
mainDF = df[['nickName', 'foundationDate']]
mainDF = mainDF.drop_duplicates(subset=['nickName'])
mainDF['foundationDate'] = mainDF['foundationDate'].str.slice(0,10)
mainDF = mainDF[mainDF['foundationDate'] != None] #this is not working
mainDF

,nickName,foundationDate
0,Well Teste,2017-02-07
1,VP Produtos Digitais,2020-01-01
2,MCo,None
5,BBB Banheiras,None
6,Mercearia Aliança,None
...,...,...
995,MINI DELICIAS LOJA 2,2020-06-16
996,Kid's Brasil,2016-08-25
997,CONSULTORIA HARMONIA,2020-01-30
998,TO BE CURSOS ONLINE,2020-07-17


In [705]:
from datetime import datetime
age = []
now = datetime.now()
for date in mainDF['foundationDate']:
    if date==None:
        datetimeObject = None
        age.append(datetimeObject)
    else:
        datetimeObject = datetime.strptime(date, '%Y-%m-%d')
        diff = now - datetimeObject
        age.append(diff.days/365)
mainDF['Age (Years)'] = age
mainDF.to_json('/Users/alizia/AliDivi/proof-of-concept/datasets/original_bc_data.json')
mainDF

,nickName,foundationDate,Age (Years)
0,Well Teste,2017-02-07,6.526027
1,VP Produtos Digitais,2020-01-01,3.627397
2,MCo,None,NaN
5,BBB Banheiras,None,NaN
6,Mercearia Aliança,None,NaN
...,...,...,...
995,MINI DELICIAS LOJA 2,2020-06-16,3.169863
996,Kid's Brasil,2016-08-25,6.980822
997,CONSULTORIA HARMONIA,2020-01-30,3.547945
998,TO BE CURSOS ONLINE,2020-07-17,3.084932


After conversion to age in years, we can build our age rejection model. Since the actual complex analysis of the plan requires a legitimate model and these are only swift rejection parameters, we can have a set of if statements for conditions like age that make rejections before the model is even ran:

In [706]:
approved = []
for age in mainDF['Age (Years)']:
    if age <= .5:
        approved.append(False)
    else:
        approved.append(None)
mainDF['Approved'] = approved
mainDF

,nickName,foundationDate,Age (Years),Approved
0,Well Teste,2017-02-07,6.526027,None
1,VP Produtos Digitais,2020-01-01,3.627397,None
2,MCo,None,NaN,None
5,BBB Banheiras,None,NaN,None
6,Mercearia Aliança,None,NaN,None
...,...,...,...,...
995,MINI DELICIAS LOJA 2,2020-06-16,3.169863,None
996,Kid's Brasil,2016-08-25,6.980822,None
997,CONSULTORIA HARMONIA,2020-01-30,3.547945,None
998,TO BE CURSOS ONLINE,2020-07-17,3.084932,None


Then, we can see if the company has reported a loss based on the average growth percentage. This was provided by the work done by Gian and Ana on creating a excel file with our previous clients and their revenue every month. Because I am using two different data sets, I had to do these tests separately, but in principle the actual model I make will require ONE data set to have all of these parameters.

In [707]:
xl = pd.ExcelFile("/Users/alizia/AliDivi/proof-of-concept/datasets/averageGrowth.xlsx")
df = xl.parse("TRABALHADO2", header=0)
df

,Record ID,Company name,City,Country/Region,Segment,Type,CNPJ,Reported Revenue,Initial Reported Revenue,Latest Reported Revenue,...,Months in base,Growth / Mo,Growth (AV) / Mo,Unnamed: 19,Unnamed: 20,EC,NC,Unnamed: 23,Unnamed: 24,Unnamed: 25
0,10263421458,Solo Snacks,NaN,Brazil,Alimentos / Bebidas (varejo),Client,40051319000160,137343.39,137343.39,257922.21,...,4,0.116875,0.120259,NaN,Mean,-0.159298,-0.428669,NaN,-0.015958,-0.03291
1,8231073880,Seven Lançamentos,NaN,NaN,Outros,Client,37146885000141,100363.71,100363.71,165494.21,...,4,0.098388,0.063940,NaN,Median,NaN,NaN,NaN,NaN,NaN
2,10325605546,Yamo,NaN,NaN,Alimentos / Bebidas (varejo),Client,38492760000136,98469.48,98469.48,38340.52,...,4,-0.392072,-0.134637,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,8763648616,Grape Time,NaN,NaN,Alimentos / Bebidas (varejo),Client,24123113000142,12736.79,12736.79,9780.00,...,4,-0.075583,-0.184604,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,8230926495,4k Cosmetics,NaN,Brazil,Beleza e cuidados pessoais (varejo),Client,33113626000163,24751.71,24751.71,22678.91,...,4,-0.022849,0.000426,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109,8230801506,Fokus Fit,NaN,Brazil,Bem estar,Client,24255324000139,941925.01,941925.01,1205164.44,...,25,0.008737,0.009369,NaN,NaN,NaN,NaN,NaN,NaN,NaN
110,7844006753,Lavemcasa,Belo Horizonte,Brazil,Serviços profissionais,Client,23517943000191,12762.98,12762.98,23895.84,...,25,0.018636,0.018848,NaN,NaN,NaN,NaN,NaN,NaN,NaN
111,7843998991,E-Wines,Planaltina,NaN,Alimentos / Bebidas (varejo),Client,37484628000110,161484.07,161484.07,124119.78,...,26,-0.011578,-0.014658,NaN,NaN,NaN,NaN,NaN,NaN,NaN
112,7843998990,Nutripro - Foodz,São Paulo,Brazil,Alimentos / Bebidas (varejo),Client,33544436000109,13768.71,13768.71,179648.56,...,27,0.034198,0.034860,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [708]:
df = df.sort_values("Company name")
df

,Record ID,Company name,City,Country/Region,Segment,Type,CNPJ,Reported Revenue,Initial Reported Revenue,Latest Reported Revenue,...,Months in base,Growth / Mo,Growth (AV) / Mo,Unnamed: 19,Unnamed: 20,EC,NC,Unnamed: 23,Unnamed: 24,Unnamed: 25
61,8365661347,100 Foods,NaN,Brazil,Alimentos / Bebidas (varejo),Client,27997623000137,93069.00,93069.00,64550.36,...,10,-0.044180,-0.033487,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50,8231020686,1Recado,NaN,Brazil,Entretenimento,Client,34424334000104,14045.92,14045.92,12489.39,...,8,-0.015579,0.007950,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,8230926495,4k Cosmetics,NaN,Brazil,Beleza e cuidados pessoais (varejo),Client,33113626000163,24751.71,24751.71,22678.91,...,4,-0.022849,0.000426,NaN,NaN,NaN,NaN,NaN,NaN,NaN
113,7844017189,Advbox,Florianópolis,Brazil,Serviços profissionais,Client,30750260000145,193023.15,193023.15,1059313.05,...,27,0.030288,0.030157,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75,7844122539,Agencia Lead,São Paulo,Brazil,Marketing / Agências,Client,31410323000187,19673.92,19673.92,1000000.00,...,12,0.081694,0.078647,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47,8361891733,Yes,NaN,Brazil,Casa e decoração (varejo),Client,26624044000186,378691.98,378691.98,96776.88,...,8,-0.364130,-0.241799,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100,7844106753,Yori Cosméticos,Florianópolis,Brazil,Beleza e cuidados pessoais (varejo),Client,32615057000191,10456.62,10456.62,35163.14,...,18,0.039035,0.037081,NaN,NaN,NaN,NaN,NaN,NaN,NaN
28,8517524538,Youbecome,NaN,NaN,Alimentos / Bebidas (varejo),Client,25464755000178,77415.00,77415.00,41363.25,...,6,-0.145265,-0.079205,NaN,NaN,NaN,NaN,NaN,NaN,NaN
36,8541798492,basico.com,Sao Paulo,Brazil,Moda (varejo),Client,41283864000145,2469735.62,2469735.62,2070662.10,...,7,-0.027532,-0.007387,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [709]:
resultDF = df[df['Growth (AV) / Mo'] > 0]
resultDF

,Record ID,Company name,City,Country/Region,Segment,Type,CNPJ,Reported Revenue,Initial Reported Revenue,Latest Reported Revenue,...,Months in base,Growth / Mo,Growth (AV) / Mo,Unnamed: 19,Unnamed: 20,EC,NC,Unnamed: 23,Unnamed: 24,Unnamed: 25
50,8231020686,1Recado,NaN,Brazil,Entretenimento,Client,34424334000104,14045.92,14045.92,12489.39,...,8,-0.015579,0.007950,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,8230926495,4k Cosmetics,NaN,Brazil,Beleza e cuidados pessoais (varejo),Client,33113626000163,24751.71,24751.71,22678.91,...,4,-0.022849,0.000426,NaN,NaN,NaN,NaN,NaN,NaN,NaN
113,7844017189,Advbox,Florianópolis,Brazil,Serviços profissionais,Client,30750260000145,193023.15,193023.15,1059313.05,...,27,0.030288,0.030157,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75,7844122539,Agencia Lead,São Paulo,Brazil,Marketing / Agências,Client,31410323000187,19673.92,19673.92,1000000.00,...,12,0.081694,0.078647,NaN,NaN,NaN,NaN,NaN,NaN,NaN
45,8230970791,Ativum Contabilidade,NaN,Brazil,Serviços profissionais,Client,29402935000186,41985.00,41985.00,72608.10,...,8,0.052720,0.049158,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,7844017190,Auba,São Paulo,Brazil,Marketing / Agências,Client,13611634000122,1270.00,1270.00,970.00,...,4,-0.077320,0.046474,NaN,NaN,NaN,NaN,NaN,NaN,NaN
96,8365874698,Awari,São Paulo,Brazil,Educação / Treinamentos,Client,32310142000141,678528.77,678528.77,747178.97,...,17,0.005405,0.008495,NaN,NaN,NaN,NaN,NaN,NaN,NaN
52,8365858856,B-On Saudável,NaN,Brazil,Bem estar,Client,24955190000169,17572.86,17572.86,35714.68,...,8,0.063496,0.046625,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17,7844106681,Beauty Foco,Guarulhos,NaN,Beleza e cuidados pessoais (varejo),Client,40151290000199,2538.53,2538.53,8695.02,...,5,0.141610,0.189479,NaN,NaN,NaN,NaN,NaN,NaN,NaN
71,7843998987,Bevenuto,Campinas,Brazil,Moda (varejo),Client,37626243000140,9073.37,9073.37,80343.54,...,11,0.080643,0.076023,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Just by cleaning our data, we can see how to deduce the information needed for our first two parameters. However, this is not going to suffice for our situation where we need the CNPJ, age, avg growth /month, all in one table so that we can add a column for a True/False approved column with which to feed to our final model. So, we need to combine some existing tables provided by Ana and Gian.

In [710]:
otherHalfDF = pd.read_csv("/Users/alizia/AliDivi/proof-of-concept/datasets/Ali_Client_Info_v5.csv")
otherHalfDF

,account_nickname,cnpj,ccb_number,origin,foundation_date
0,AUTO POSITION RASTREAMENTO,23104098000122,A0624639-000,Inbound lead,8/20/2015
1,100 Foods,27997623000137,A1100640-000,Outbound sales,2/14/2017
2,1Recado,34424334000104,A1237320-000,Inbound lead,8/5/2019
3,4f life & business,6311283000179,A0746811-000,Inbound lead,6/7/2004
4,4k Cosmetics,33113626000163,A1319793-000,Inbound lead,3/22/2019
...,...,...,...,...,...
205,Ziel Cosméticos,41670524000177,A2201237-000,Outbound sales,4/22/2021
206,gabi games comercio eletronico ltda,47233207000170,A2959889-000,Outbound sales,7/21/2022
207,iKEG,29973569000115,A0585828-000,Inbound lead,3/19/2018
208,Ânimo,35081741000129,A0707800-000,Outbound sales,12/13/2019


## Merging Tables to Finalize Data

We need to combine the two tables, Ali_Client_Info and averageGrowth, into one table so that our simple model has all the parameters it needs. The CNPJ is common among both of these but its in sci not in Ali_Client_Info. That's okay, because that's just how float64s are printed in Python. We can still do direct int to float comparisons. First, I'll print out both tables to see what we are merging

In [711]:
df = df.astype({"CNPJ": str})
df

,Record ID,Company name,City,Country/Region,Segment,Type,CNPJ,Reported Revenue,Initial Reported Revenue,Latest Reported Revenue,...,Months in base,Growth / Mo,Growth (AV) / Mo,Unnamed: 19,Unnamed: 20,EC,NC,Unnamed: 23,Unnamed: 24,Unnamed: 25
61,8365661347,100 Foods,NaN,Brazil,Alimentos / Bebidas (varejo),Client,27997623000137,93069.00,93069.00,64550.36,...,10,-0.044180,-0.033487,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50,8231020686,1Recado,NaN,Brazil,Entretenimento,Client,34424334000104,14045.92,14045.92,12489.39,...,8,-0.015579,0.007950,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,8230926495,4k Cosmetics,NaN,Brazil,Beleza e cuidados pessoais (varejo),Client,33113626000163,24751.71,24751.71,22678.91,...,4,-0.022849,0.000426,NaN,NaN,NaN,NaN,NaN,NaN,NaN
113,7844017189,Advbox,Florianópolis,Brazil,Serviços profissionais,Client,30750260000145,193023.15,193023.15,1059313.05,...,27,0.030288,0.030157,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75,7844122539,Agencia Lead,São Paulo,Brazil,Marketing / Agências,Client,31410323000187,19673.92,19673.92,1000000.00,...,12,0.081694,0.078647,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47,8361891733,Yes,NaN,Brazil,Casa e decoração (varejo),Client,26624044000186,378691.98,378691.98,96776.88,...,8,-0.364130,-0.241799,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100,7844106753,Yori Cosméticos,Florianópolis,Brazil,Beleza e cuidados pessoais (varejo),Client,32615057000191,10456.62,10456.62,35163.14,...,18,0.039035,0.037081,NaN,NaN,NaN,NaN,NaN,NaN,NaN
28,8517524538,Youbecome,NaN,NaN,Alimentos / Bebidas (varejo),Client,25464755000178,77415.00,77415.00,41363.25,...,6,-0.145265,-0.079205,NaN,NaN,NaN,NaN,NaN,NaN,NaN
36,8541798492,basico.com,Sao Paulo,Brazil,Moda (varejo),Client,41283864000145,2469735.62,2469735.62,2070662.10,...,7,-0.027532,-0.007387,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [712]:
otherHalfDF

,account_nickname,cnpj,ccb_number,origin,foundation_date
0,AUTO POSITION RASTREAMENTO,23104098000122,A0624639-000,Inbound lead,8/20/2015
1,100 Foods,27997623000137,A1100640-000,Outbound sales,2/14/2017
2,1Recado,34424334000104,A1237320-000,Inbound lead,8/5/2019
3,4f life & business,6311283000179,A0746811-000,Inbound lead,6/7/2004
4,4k Cosmetics,33113626000163,A1319793-000,Inbound lead,3/22/2019
...,...,...,...,...,...
205,Ziel Cosméticos,41670524000177,A2201237-000,Outbound sales,4/22/2021
206,gabi games comercio eletronico ltda,47233207000170,A2959889-000,Outbound sales,7/21/2022
207,iKEG,29973569000115,A0585828-000,Inbound lead,3/19/2018
208,Ânimo,35081741000129,A0707800-000,Outbound sales,12/13/2019


In [713]:
df1 = df
df2 = otherHalfDF.rename(columns={'cnpj':'CNPJ'})
df2 = df2.astype({"CNPJ": str})
df2

,account_nickname,CNPJ,ccb_number,origin,foundation_date
0,AUTO POSITION RASTREAMENTO,23104098000122,A0624639-000,Inbound lead,8/20/2015
1,100 Foods,27997623000137,A1100640-000,Outbound sales,2/14/2017
2,1Recado,34424334000104,A1237320-000,Inbound lead,8/5/2019
3,4f life & business,6311283000179,A0746811-000,Inbound lead,6/7/2004
4,4k Cosmetics,33113626000163,A1319793-000,Inbound lead,3/22/2019
...,...,...,...,...,...
205,Ziel Cosméticos,41670524000177,A2201237-000,Outbound sales,4/22/2021
206,gabi games comercio eletronico ltda,47233207000170,A2959889-000,Outbound sales,7/21/2022
207,iKEG,29973569000115,A0585828-000,Inbound lead,3/19/2018
208,Ânimo,35081741000129,A0707800-000,Outbound sales,12/13/2019


In [714]:
joined_frame = df2.merge(df1, on='CNPJ', how='left')
finalData = joined_frame
finalData = joined_frame[['account_nickname', 'CNPJ', 'foundation_date', 'Growth (AV) / Mo']].copy()
finalData = finalData.rename(columns={'CNPJ':'cnpj', 'Growth (AV) / Mo':'avg_growth_per_month'})
finalData.to_json('/Users/alizia/AliDivi/proof-of-concept/datasets/original_bc_data.json')

finalData

,account_nickname,cnpj,foundation_date,avg_growth_per_month
0,AUTO POSITION RASTREAMENTO,23104098000122,8/20/2015,NaN
1,100 Foods,27997623000137,2/14/2017,-0.033487
2,1Recado,34424334000104,8/5/2019,0.007950
3,4f life & business,6311283000179,6/7/2004,NaN
4,4k Cosmetics,33113626000163,3/22/2019,0.000426
...,...,...,...,...
205,Ziel Cosméticos,41670524000177,4/22/2021,NaN
206,gabi games comercio eletronico ltda,47233207000170,7/21/2022,NaN
207,iKEG,29973569000115,3/19/2018,-0.536123
208,Ânimo,35081741000129,12/13/2019,NaN


Now that things are merged, we can convert all of our foundation dates to ages. Thanks to Vini's help with some python shortcuts and Ana's help with filling in null values, I was able to get this working!

In [715]:
finalData.foundation_date.unique()

array(['8/20/2015', '2/14/2017', '8/5/2019', '6/7/2004', '3/22/2019',
       '6/20/2018', '8/28/2019', '8/24/2018', '8/20/2020', '1/7/2017',
       '5/5/2014', '1/10/2018', '4/20/2011', '12/26/2018', '5/2/2016',
       '7/14/2016', '9/23/2013', '12/18/2020', '2/27/2020', '8/19/2020',
       '7/6/2020', '6/10/2005', '2/17/2021', '3/10/2017', '4/6/2020',
       '8/26/2020', '3/19/2021', '3/12/2017', '1/8/2021', '3/28/2022',
       '4/3/2017', '5/2/2022', '8/11/2015', '8/17/2020', '6/19/2019',
       '1/30/2020', '12/16/2016', '6/16/2020', '6/20/2017', '7/4/2018',
       '2/1/2019', '3/24/2014', '3/6/2012', '9/19/2007', nan,
       '10/29/2019', '9/9/2016', '7/24/2019', '7/10/2014', '8/3/2018',
       '1/19/2021', '3/27/2012', '8/2/2011', '2/11/2019', '9/24/2018',
       '5/8/2015', '1/5/2016', '11/22/2010', '6/22/2020', '7/2/2020',
       '7/26/2015', '10/26/2011', '6/6/2011', '4/3/2019', '9/4/2017',
       '5/3/2016', '1/8/2013', '1/25/2016', '9/3/2020', '9/17/2014',
       '7/9/2020', 

In [716]:
test = finalData['foundation_date']
test.at[46] = "09/28/2017"
test.at[187] = "02/19/2013"
test.at[190] = "01/05/2018"
test = pd.to_datetime(finalData['foundation_date'], format='%m/%d/%Y')
test[test.isna()]

Series([], Name: foundation_date, dtype: datetime64[ns])

In [717]:
test[test.isna()]

Series([], Name: foundation_date, dtype: datetime64[ns])

In [718]:
test = (datetime.now() - test) / np.timedelta64(1, 'Y')
test

0       7.994359
1       6.504938
2       4.035345
3      19.195136
4       4.407701
         ...    
205     2.321416
206     1.075668
207     5.415251
208     3.679417
209     6.288643
Name: foundation_date, Length: 210, dtype: float64

In [719]:
finalData['ages'] = test
finalData

,account_nickname,cnpj,foundation_date,avg_growth_per_month,ages
0,AUTO POSITION RASTREAMENTO,23104098000122,8/20/2015,NaN,7.994359
1,100 Foods,27997623000137,2/14/2017,-0.033487,6.504938
2,1Recado,34424334000104,8/5/2019,0.007950,4.035345
3,4f life & business,6311283000179,6/7/2004,NaN,19.195136
4,4k Cosmetics,33113626000163,3/22/2019,0.000426,4.407701
...,...,...,...,...,...
205,Ziel Cosméticos,41670524000177,4/22/2021,NaN,2.321416
206,gabi games comercio eletronico ltda,47233207000170,7/21/2022,NaN,1.075668
207,iKEG,29973569000115,3/19/2018,-0.536123,5.415251
208,Ânimo,35081741000129,12/13/2019,NaN,3.679417


In [720]:
final_data = finalData
final_data = final_data.drop(columns = ['foundation_date'])
final_data = final_data.rename(columns={'ages':'age'})
final_data

,account_nickname,cnpj,avg_growth_per_month,age
0,AUTO POSITION RASTREAMENTO,23104098000122,NaN,7.994359
1,100 Foods,27997623000137,-0.033487,6.504938
2,1Recado,34424334000104,0.007950,4.035345
3,4f life & business,6311283000179,NaN,19.195136
4,4k Cosmetics,33113626000163,0.000426,4.407701
...,...,...,...,...
205,Ziel Cosméticos,41670524000177,NaN,2.321416
206,gabi games comercio eletronico ltda,47233207000170,NaN,1.075668
207,iKEG,29973569000115,-0.536123,5.415251
208,Ânimo,35081741000129,NaN,3.679417


Since this is based off a database of previous Divi clients, none of these clients would have had less than 6 months of age, so these wont reflect in the data fed to the model. The best way to avoid this is to have a simple age checker running right before a prediction is ran on the company. Doing it this way is much better for performance as well, since the model's prediction doesnt have to run for a company that would have never been approved in the first place.

In [721]:
final_data['approved'] = (final_data.avg_growth_per_month > 0)
final_data

,account_nickname,cnpj,avg_growth_per_month,age,approved
0,AUTO POSITION RASTREAMENTO,23104098000122,NaN,7.994359,False
1,100 Foods,27997623000137,-0.033487,6.504938,False
2,1Recado,34424334000104,0.007950,4.035345,True
3,4f life & business,6311283000179,NaN,19.195136,False
4,4k Cosmetics,33113626000163,0.000426,4.407701,True
...,...,...,...,...,...
205,Ziel Cosméticos,41670524000177,NaN,2.321416,False
206,gabi games comercio eletronico ltda,47233207000170,NaN,1.075668,False
207,iKEG,29973569000115,-0.536123,5.415251,False
208,Ânimo,35081741000129,NaN,3.679417,False


This is far from the final data set, but it provides enough for a dummy model. Let's save it to a JSON.

In [722]:
final_data.to_json('/Users/alizia/AliDivi/proof-of-concept/datasets/dummyData.json')

In [723]:
dummyData = pd.read_json('/Users/alizia/AliDivi/proof-of-concept/datasets/dummyData.json')
dummyData

,account_nickname,cnpj,avg_growth_per_month,age,approved
0,AUTO POSITION RASTREAMENTO,23104098000122,NaN,7.994359,False
1,100 Foods,27997623000137,-0.033487,6.504938,False
2,1Recado,34424334000104,0.007950,4.035345,True
3,4f life & business,6311283000179,NaN,19.195136,False
4,4k Cosmetics,33113626000163,0.000426,4.407701,True
...,...,...,...,...,...
205,Ziel Cosméticos,41670524000177,NaN,2.321416,False
206,gabi games comercio eletronico ltda,47233207000170,NaN,1.075668,False
207,iKEG,29973569000115,-0.536123,5.415251,False
208,Ânimo,35081741000129,NaN,3.679417,False


### Done with our basic data! Now we can move on to simple predictions.

The following few lines just pinpoint where the null growth values are so that I could easily ask Ana about where to find them for the future

In [724]:
problems = dummyData[dummyData['avg_growth_per_month'].isna()]
problems

,account_nickname,cnpj,avg_growth_per_month,age,approved
0,AUTO POSITION RASTREAMENTO,23104098000122,NaN,7.994359,False
3,4f life & business,6311283000179,NaN,19.195136,False
6,AGENCIA FIRE,34695214000141,NaN,3.972374,False
9,ATABUA COMERCIO GOURMET LTDA,29009901000126,NaN,6.608978,False
10,Allya,20182828000115,NaN,9.286651,False
...,...,...,...,...,...
200,YENZAH,23077168000109,NaN,7.153821,False
204,Z2,41255434000110,NaN,2.419980,False
205,Ziel Cosméticos,41670524000177,NaN,2.321416,False
206,gabi games comercio eletronico ltda,47233207000170,NaN,1.075668,False


In [725]:
problems.to_excel("output.xlsx")

In [726]:
dummyData = dummyData[dummyData['avg_growth_per_month'].isna()==False]
dummyData.to_json('/Users/alizia/AliDivi/proof-of-concept/datasets/dummyDataFinal.json')
dummyData

,account_nickname,cnpj,avg_growth_per_month,age,approved
1,100 Foods,27997623000137,-0.033487,6.504938,False
2,1Recado,34424334000104,0.007950,4.035345,True
4,4k Cosmetics,33113626000163,0.000426,4.407701,True
5,ADVBOX,30750260000145,0.030157,5.160625,True
7,AGENCIA LEAD,31410323000187,0.078647,4.982661,True
...,...,...,...,...,...
201,YOUBECOME,25464755000178,-0.079205,7.016926,False
202,Yes We Grow,26624044000186,-0.241799,6.715756,False
203,Yori Cosméticos e Beleza,32615057000191,0.037081,4.547334,True
207,iKEG,29973569000115,-0.536123,5.415251,False


We are going to work with the data that actually has avergae growth per month information.

#### Train-Test Split (we want to obtain the 'approved' variable)

In [727]:
from sklearn.model_selection import train_test_split
y = dummyData['approved']
X = dummyData[['avg_growth_per_month', 'age']]

In [728]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=100)

In [729]:
X_train

,avg_growth_per_month,age
116,-0.063628,7.493322
17,0.189479,2.663654
65,-0.130415,4.374846
175,-0.025314,3.131836
173,0.012282,4.276281
...,...,...
164,-0.000160,5.261928
188,0.051989,19.285487
123,0.019144,3.402889
42,-0.084273,4.541858


In [730]:
X_test

,avg_growth_per_month,age
177,0.032134,2.017508
181,-0.451456,3.320752
199,-0.134637,2.915541
48,0.114477,6.937527
190,0.023755,5.615118
61,-0.011374,3.126360
201,-0.079205,7.016926
56,-0.025467,4.897786
78,0.009369,7.476895
39,0.019079,6.159961


In [731]:
from sklearn.linear_model import LinearRegression
lr = LinearRegression()
lr.fit(X_train, y_train)
LinearRegression()

LinearRegression()

In [732]:
y_lr_train_pred = lr.predict(X_train)
y_lr_test_pred = lr.predict(X_test)

In [733]:
y_lr_train_pred

array([ 0.44191654,  1.10237998,  0.30473835,  0.57255269,  0.65493221,
        0.40685682,  0.67424984,  0.45130434,  0.72440805,  0.7934938 ,
        0.57301478,  0.32124714,  0.48029936,  0.44618889,  0.73402501,
        0.69122672,  0.57553191,  0.42015645,  0.51804443,  0.52548963,
        0.70434542,  0.70141836,  0.51312666,  0.63269963,  0.66251161,
        0.71016449,  0.35687929,  0.55519149,  0.93872824,  0.52062393,
        0.62250096,  0.74218742,  0.64184191,  0.5458307 ,  0.71270354,
        0.52405728,  0.86062962,  0.58113467,  0.89070582,  0.14538676,
        0.70200906, -1.10551527,  0.75193714,  0.62179235,  0.64735015,
        0.74947184, -0.14876452,  0.64106571,  0.22539425,  0.80566103,
        0.52927363,  0.58790444,  0.48067458,  0.59810732,  0.37932792,
        0.47098282,  0.30509453,  0.67064343,  0.71335888,  0.69320064,
        0.93267848,  0.87239527,  0.82079361,  0.64636213,  0.59845569,
        0.8114466 ,  0.63728733,  0.62479762,  0.50093319,  0.71

Model evaluation

In [734]:
from sklearn.metrics import mean_squared_error, r2_score
lr_train_mse = mean_squared_error(y_train, y_lr_train_pred)
lr_train_r2 = r2_score(y_train, y_lr_train_pred)

lr_test_mse = mean_squared_error(y_test, y_lr_test_pred)
lr_test_r2 = r2_score(y_test, y_lr_test_pred)

In [735]:
lr_train_mse

0.15686584604293977

In [736]:
lr_train_r2

0.36633850191142237

In [737]:
lr_test_mse

0.16811772638517664

In [738]:
lr_test_r2

0.3158901749403197

Relatively similar results when training the model and asking it to predict what is already known to us in the test data. Let's try pearson correlation coefficient!

The following code is an attempt to fix some of the empty average growth per month data points from the previous model to have a better start when feeding the model. I feel like it makes more sense to solve this problem before trying more sklearn models.

In [739]:
exl = pd.ExcelFile("/Users/alizia/AliDivi/proof-of-concept/datasets/Ali_Info.xlsx")
missingAvgGrowth = exl.parse("Sheet1", header=0)
missingAvgGrowth = missingAvgGrowth.sort_values("account_nickname")
missingAvgGrowth

,cnpj,Average of RR (Reported Revenue),Average of Initial RR,Average of Recent RR,growth_overall,account_nickname
57,23104098000122,1.224328e+05,101795.00,140000.00,0.272893,AUTO POSITION RASTREAMENTO
87,27997623000137,6.970888e+04,93069.00,64550.36,-0.441805,100 Foods
134,34424334000104,1.776868e+04,14045.92,12489.39,-0.124628,1Recado
6,6311283000179,1.942909e+04,5595.00,51113.66,0.890538,4f life & business
123,33113626000163,2.478343e+04,24751.71,22678.91,-0.091398,4k Cosmetics
...,...,...,...,...,...,...
184,41255434000110,1.106212e+06,1106212.29,1106212.29,0.000000,Z2
186,41670524000177,1.954846e+04,13016.22,26080.70,0.500925,Ziel CosmÃ©ticos
97,29973569000115,1.966918e+05,717750.56,103567.30,-5.930282,iKEG
141,35081741000129,9.914886e+03,755.77,18713.97,0.959615,Ã‚nimo


In [740]:
joined_growth_overall_with_avg = dummyData.merge(missingAvgGrowth, on='cnpj', how='left')
joined_growth_overall_with_avg

,account_nickname_x,cnpj,avg_growth_per_month,age,approved,Average of RR (Reported Revenue),Average of Initial RR,Average of Recent RR,growth_overall,account_nickname_y
0,100 Foods,27997623000137,-0.033487,6.504938,False,69708.879000,93069.00,64550.36,-0.441805,100 Foods
1,1Recado,34424334000104,0.007950,4.035345,True,17768.677500,14045.92,12489.39,-0.124628,1Recado
2,4k Cosmetics,33113626000163,0.000426,4.407701,True,24783.430000,24751.71,22678.91,-0.091398,4k Cosmetics
3,ADVBOX,30750260000145,0.030157,5.160625,True,564349.736667,193023.15,1059313.05,0.817785,ADVBOX
4,AGENCIA LEAD,31410323000187,0.078647,4.982661,True,113822.700000,19673.92,1000000.00,0.980326,AGENCIA LEAD
...,...,...,...,...,...,...,...,...,...,...
109,YOUBECOME,25464755000178,-0.079205,7.016926,False,60374.173333,77415.00,41363.25,-0.871589,YOUBECOME
110,Yes We Grow,26624044000186,-0.241799,6.715756,False,207515.513750,378691.98,96776.88,-2.913042,Yes We Grow
111,Yori Cosméticos e Beleza,32615057000191,0.037081,4.547334,True,27449.134444,10456.62,35163.14,0.702626,Yori CosmÃ©ticos e Beleza
112,iKEG,29973569000115,-0.536123,5.415251,False,196691.803077,717750.56,103567.30,-5.930282,iKEG


In [741]:
compDummyData = joined_growth_overall_with_avg.drop(columns = ['account_nickname_y'])
compDummyData = compDummyData.reindex(columns=['account_nickname_x', 'cnpj', 'age', 'growth_overall', 'approved'])
compDummyData['approved'] = (compDummyData.growth_overall > 0)
compDummyData

,account_nickname_x,cnpj,age,growth_overall,approved
0,100 Foods,27997623000137,6.504938,-0.441805,False
1,1Recado,34424334000104,4.035345,-0.124628,False
2,4k Cosmetics,33113626000163,4.407701,-0.091398,False
3,ADVBOX,30750260000145,5.160625,0.817785,True
4,AGENCIA LEAD,31410323000187,4.982661,0.980326,True
...,...,...,...,...,...
109,YOUBECOME,25464755000178,7.016926,-0.871589,False
110,Yes We Grow,26624044000186,6.715756,-2.913042,False
111,Yori Cosméticos e Beleza,32615057000191,4.547334,0.702626,True
112,iKEG,29973569000115,5.415251,-5.930282,False


Great! We successfully cleared any null values and went back to the overall growth as our metric. Now, we can use Logistic Regression to find the relation between numbers and booleans (approved or not approved)

In [742]:
X = compDummyData.drop(columns = ['account_nickname_x', 'cnpj', 'approved'])
y = compDummyData['approved']

In [743]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state=21)

In [744]:
X_train

,age,growth_overall
42,10.661080,-0.497344
64,7.493322,-0.338033
27,4.068200,0.172176
46,8.585747,-0.189416
1,4.035345,-0.124628
...,...,...
48,3.613708,0.648933
4,4.982661,0.980326
56,8.144944,0.200992
79,3.942257,-0.765879


In [745]:
from sklearn.preprocessing import StandardScaler

In [746]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.fit_transform(X_test)
#we dont feed the test data to the model, only the train

In [747]:
from sklearn.linear_model import LogisticRegression
log_reg = LogisticRegression(random_state = 0).fit(X_train_scaled, y_train)
log_reg.predict(X_train_scaled)

array([ True,  True,  True,  True, False,  True, False,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
       False, False,  True,  True,  True, False, False,  True, False,
        True,  True,  True,  True,  True,  True,  True, False,  True,
        True,  True, False, False,  True, False, False,  True,  True,
        True, False, False,  True, False,  True, False,  True,  True,
       False,  True, False,  True, False, False,  True, False, False,
        True,  True, False, False,  True,  True,  True,  True,  True,
       False, False,  True,  True,  True, False,  True])

In [748]:
log_reg.score(X_train_scaled, y_train)

0.7721518987341772

In [749]:
log_reg.score(X_test_scaled, y_test)

0.6857142857142857

This model is only decent since there is a big mismatch in train and test scores.

In [750]:
log_reg = LogisticRegression(random_state = 0, C = 1, fit_intercept = True).fit(X_train_scaled, y_train)

In [751]:
log_reg.score(X_train_scaled, y_train)

0.7721518987341772

In [752]:
log_reg.score(X_test_scaled, y_test)

0.6857142857142857

After removing outliers and choosing to account for the intercept (C=1 and fit_intercept), the model is more likely to improve in its score when tested among larger data sets. The logistic regression model proves to be quite decent for basic statements on auto rejection, but it definitely needs more data and more parameters. For the next iteration, we are gonna see if we can retrive credit buereau data on whether or not companies already owe money or not. This is another criteria for swift rejection.

For the meantime, let's make some fake companies and see how the model predicts them

In [753]:
data = {'name': ['Bobs Burgers', 'Chicken Brothers', 'Moleque Zica'], 'age': [3, 3, 19], 'growth_overall': [-24.56, 4.56, 1.97]}  
fake_data = pd.DataFrame(data)
fake_data
# fake_data_scaled = scaler.fit_transform(X_train)
# log_reg.predict()

,name,age,growth_overall
0,Bobs Burgers,3,-24.56
1,Chicken Brothers,3,4.56
2,Moleque Zica,19,1.97


In [754]:
fake_data_scaled = scaler.fit_transform(fake_data.drop(columns=['name']))
result = log_reg.predict(fake_data_scaled)

In [755]:
fake_data['approved'] = result
fake_data

,name,age,growth_overall,approved
0,Bobs Burgers,3,-24.56,False
1,Chicken Brothers,3,4.56,True
2,Moleque Zica,19,1.97,True


Let's save this very basic model with pickle!

In [756]:
import pickle
with open('/Users/alizia/AliDivi/proof-of-concept/models/model0.pkl','wb') as f:
    pickle.dump(log_reg,f)

## Second Iteration

Now that we have a semi-functioning basic rejection model, we can tack on data from Superset query to see how the info we have right now coincides with how much money the company was offered.

First, we can try to access Serasa and process whether a company has a current deliquency or not.

In [757]:
comp_clients_df = pd.read_csv('/Users/alizia/AliDivi/proof-of-concept/datasets/comp_past_clients.csv')
comp_clients_df = comp_clients_df.drop_duplicates(subset=['account_nickname'])
comp_clients_df = comp_clients_df.sort_values("account_nickname")
comp_clients_df

,account_nickname,cnpj,ccb_number,release_date,gross_value,net_value,revenue_share,yearly_cet,monthly_cet,yearly_interest_rate,monthly_interest_rate,tac_rate,tac_value,iof_rate,iof_value,rating,origin
0,AUTO POSITION RASTREAMENTO,23104098000122,A0624639-000,2021-01-28,10166.11,10000.00,1.6,32.92,2.40,29.080444,2.15,1.000000,100.00,0.006546,66.1100,B,Inbound lead
3,100 Foods,27997623000137,A1100640-000,2021-11-26,10367.31,10000.00,2.0,35.75,2.58,26.824179,2.00,3.000000,300.00,0.006535,67.3100,B,Outbound sales
8,1Recado,34424334000104,A1237320-000,2022-01-31,10367.42,10000.00,7.0,35.60,2.57,26.824179,2.00,3.000000,300.00,0.006546,67.4200,B,Inbound lead
9,4f life & business,6311283000179,A0746811-000,2021-05-28,1056.90,1000.00,0.6,46.96,3.26,32.922800,2.40,5.000000,50.00,0.006571,6.9000,CC,Inbound lead
10,4k Cosmetics,33113626000163,A1319793-000,2022-02-25,2113.86,2000.00,4.0,51.63,3.53,36.869741,2.65,5.000000,100.00,0.006600,13.8600,CC,Inbound lead
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
643,Ziel Cosméticos,41670524000177,A2201237-000,2022-10-14,35187.32,33919.99,6.2,39.61,2.82,30.604999,2.25,2.499971,847.99,0.012061,419.3400,CC,Outbound sales
644,gabi games comercio eletronico ltda,47233207000170,A2959889-000,2023-06-29,76671.01,75000.00,5.0,45.59,3.18,36.869741,2.65,1.330000,997.50,0.898013,68246.7683,CCC,Outbound sales
645,iKEG,29973569000115,A0585828-000,2020-11-26,45450.00,45000.00,0.8,0.00,2.14,26.824179,2.00,1.000000,450.00,0.000000,0.0000,BBB,Inbound lead
648,Ânimo,35081741000129,A0707800-000,2021-04-30,3157.71,3000.00,12.0,50.23,3.45,36.869741,2.65,4.000000,120.00,0.012087,37.7100,CC,Outbound sales


In [758]:
master_client_set = compDummyData.merge(comp_clients_df, on='cnpj', how='left')
master_client_set = master_client_set.drop(columns=["account_nickname", "approved"])
master_client_set = master_client_set.rename(columns={"account_nickname_x": "account_nickname"})
master_client_set


,account_nickname,cnpj,age,growth_overall,ccb_number,release_date,gross_value,net_value,revenue_share,yearly_cet,monthly_cet,yearly_interest_rate,monthly_interest_rate,tac_rate,tac_value,iof_rate,iof_value,rating,origin
0,100 Foods,27997623000137,6.504938,-0.441805,A1100640-000,2021-11-26,10367.31,10000.0,2.00,35.75,2.58,26.824179,2.00,3.000000,300.00,0.006535,67.31,B,Outbound sales
1,1Recado,34424334000104,4.035345,-0.124628,A1237320-000,2022-01-31,10367.42,10000.0,7.00,35.60,2.57,26.824179,2.00,3.000000,300.00,0.006546,67.42,B,Inbound lead
2,4k Cosmetics,33113626000163,4.407701,-0.091398,A1319793-000,2022-02-25,2113.86,2000.0,4.00,51.63,3.53,36.869741,2.65,5.000000,100.00,0.006600,13.86,CC,Inbound lead
3,ADVBOX,30750260000145,5.160625,0.817785,A0654808-000,2021-03-05,31280.26,30000.0,3.50,36.39,2.62,26.824179,2.00,3.000000,900.00,0.012306,380.26,BBB,Inbound lead
4,AGENCIA LEAD,31410323000187,4.982661,0.980326,A0745923-000,2021-05-28,5183.83,5000.0,6.00,43.41,3.05,34.488882,2.50,3.000000,150.00,0.006569,33.83,CC,Inbound lead
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109,YOUBECOME,25464755000178,7.016926,-0.871589,A1575262-000,2022-05-13,5234.15,5000.0,2.75,50.41,3.46,38.478378,2.75,4.000000,200.00,0.006567,34.15,CC,Inbound lead
110,Yes We Grow,26624044000186,6.715756,-2.913042,A1509133-000,2022-04-27,105880.59,101569.5,7.00,31.37,2.30,21.699444,1.65,3.000005,3047.09,0.012082,1264.00,BBB,Outbound sales
111,Yori Cosméticos e Beleza,32615057000191,4.547334,0.702626,A0574341-000,2020-11-13,2080.00,2000.0,1.20,0.00,3.52,40.923849,2.90,4.000000,80.00,0.000000,0.00,CC,Inbound lead
112,iKEG,29973569000115,5.415251,-5.930282,A0585828-000,2020-11-26,45450.00,45000.0,0.80,0.00,2.14,26.824179,2.00,1.000000,450.00,0.000000,0.00,BBB,Inbound lead


We now have quite a comprehensive set of data for 114 companies. Let's try to make a model that predicts the gross value (inital offer divi gives) based on age (we are gonna ignore growth rate for this because this growth metric was the growth AFTER these clients became Divi clients, so it's not a fair metric to judge our initial offer based on how the company ended up). Because this is a bare bones model, lets call it 0A, given that it's a slight rework of the previous one.

In [759]:
df_0A = master_client_set[["account_nickname", "cnpj","age", "growth_overall", "gross_value", "rating", "monthly_interest_rate"]].copy()
df_0A

,account_nickname,cnpj,age,growth_overall,gross_value,rating,monthly_interest_rate
0,100 Foods,27997623000137,6.504938,-0.441805,10367.31,B,2.00
1,1Recado,34424334000104,4.035345,-0.124628,10367.42,B,2.00
2,4k Cosmetics,33113626000163,4.407701,-0.091398,2113.86,CC,2.65
3,ADVBOX,30750260000145,5.160625,0.817785,31280.26,BBB,2.00
4,AGENCIA LEAD,31410323000187,4.982661,0.980326,5183.83,CC,2.50
...,...,...,...,...,...,...,...
109,YOUBECOME,25464755000178,7.016926,-0.871589,5234.15,CC,2.75
110,Yes We Grow,26624044000186,6.715756,-2.913042,105880.59,BBB,1.65
111,Yori Cosméticos e Beleza,32615057000191,4.547334,0.702626,2080.00,CC,2.90
112,iKEG,29973569000115,5.415251,-5.930282,45450.00,BBB,2.00


Let's also add initial reported revenue alongside age as the two input metrics

In [760]:
merge_for_rev = df[["CNPJ", "Initial Reported Revenue"]].copy()
merge_for_rev = merge_for_rev.rename(columns={"CNPJ": "cnpj","Initial Reported Revenue": "initial_reported_revenue" })
merge_for_rev

,cnpj,initial_reported_revenue
61,27997623000137,93069.00
50,34424334000104,14045.92
4,33113626000163,24751.71
113,30750260000145,193023.15
75,31410323000187,19673.92
...,...,...
47,26624044000186,378691.98
100,32615057000191,10456.62
28,25464755000178,77415.00
36,41283864000145,2469735.62


In [761]:
merge_for_rev["cnpj"] = pd.to_numeric(merge_for_rev["cnpj"])
df_0A = df_0A.merge(merge_for_rev, on='cnpj', how='left')
df_0A = df_0A.reindex(columns=['account_nickname', 'cnpj', 'age', 'growth_overall', 'initial_reported_revenue', 'rating', 'gross_value','monthly_interest_rate'])
df_0A.to_json("/Users/alizia/AliDivi/proof-of-concept/datasets/clients_given_offer.json")

df_0A


,account_nickname,cnpj,age,growth_overall,initial_reported_revenue,rating,gross_value,monthly_interest_rate
0,100 Foods,27997623000137,6.504938,-0.441805,93069.00,B,10367.31,2.00
1,1Recado,34424334000104,4.035345,-0.124628,14045.92,B,10367.42,2.00
2,4k Cosmetics,33113626000163,4.407701,-0.091398,24751.71,CC,2113.86,2.65
3,ADVBOX,30750260000145,5.160625,0.817785,193023.15,BBB,31280.26,2.00
4,AGENCIA LEAD,31410323000187,4.982661,0.980326,19673.92,CC,5183.83,2.50
...,...,...,...,...,...,...,...,...
109,YOUBECOME,25464755000178,7.016926,-0.871589,77415.00,CC,5234.15,2.75
110,Yes We Grow,26624044000186,6.715756,-2.913042,378691.98,BBB,105880.59,1.65
111,Yori Cosméticos e Beleza,32615057000191,4.547334,0.702626,10456.62,CC,2080.00,2.90
112,iKEG,29973569000115,5.415251,-5.930282,717750.56,BBB,45450.00,2.00


Now that our table has inputs (age and initial rev), we can find output (gross value and monthly interest). This is a multi output prediction, and it has the following most common models associated with it. For each iteration of this model, I will add new parameters and test all 4 of these models against the data. These are the models:

- LinearRegression (and related)
- KNeighborsRegressor
- DecisionTreeRegressor
- RandomForestRegressor (and related)
Taken from https://machinelearningmastery.com/multi-output-regression-models-with-python/

### Linear Regression for Multi-Output Credit Plan

In [762]:
from sklearn.linear_model import LinearRegression
model0A = LinearRegression()

In [763]:


X = df_0A[['age', 'growth_overall', 'initial_reported_revenue']]
y = df_0A[['gross_value','monthly_interest_rate']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=100)
model0A.fit(X_train,y_train)


LinearRegression()

In [764]:
model0A.score(X_test,y_test)

0.27770964818295724

In [765]:
#random test value for prediction
testRows = df_0A.iloc[4:8].drop(columns=["account_nickname", "rating", "cnpj", "gross_value", "monthly_interest_rate"])
testRows

,age,growth_overall,initial_reported_revenue
4,4.982661,0.980326,19673.92
5,2.992203,-0.484142,70000.00
6,5.601428,0.421759,41985.00
7,12.328466,-0.309278,1270.00


In [766]:
model0A.predict(testRows).astype(int)

array([[40864,     2],
       [31115,     2],
       [46480,     2],
       [87191,     2]])

Compare to actual value:

In [767]:
df_0A[4:8]

,account_nickname,cnpj,age,growth_overall,initial_reported_revenue,rating,gross_value,monthly_interest_rate
4,AGENCIA LEAD,31410323000187,4.982661,0.980326,19673.92,CC,5183.83,2.50
5,AGORA MARCAS,38157946000139,2.992203,-0.484142,70000.00,CC,41821.76,2.00
6,Ativum Contabilidade,29402935000186,5.601428,0.421759,41985.00,CC,5183.66,2.50
7,Auba,13611634000122,12.328466,-0.309278,1270.00,C,1594.44,3.15


Linear regression seems to be quite off....

### KNeighbors Regression for Multi-Output Credit Plan

In [768]:
from sklearn.neighbors import KNeighborsRegressor
model = KNeighborsRegressor()

In [769]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=100)
model.fit(X_train, y_train)

KNeighborsRegressor()

In [770]:
model.predict(X_test)

array([[7.18107920e+04, 2.50800000e+00],
       [7.02380000e+03, 2.66000000e+00],
       [2.92059040e+04, 2.26000000e+00],
       [3.98784400e+03, 2.80000000e+00],
       [4.08547400e+03, 2.85000000e+00],
       [6.73160788e+05, 1.83000000e+00],
       [3.03127140e+04, 2.11000000e+00],
       [3.43822180e+05, 1.87800000e+00],
       [2.44105598e+05, 1.83000000e+00],
       [6.10829720e+04, 2.16000000e+00],
       [2.29990420e+04, 2.16000000e+00],
       [7.18107920e+04, 2.50800000e+00],
       [7.82811000e+03, 2.67000000e+00],
       [4.30179360e+04, 2.37800000e+00],
       [4.30179360e+04, 2.37800000e+00],
       [3.57113214e+05, 1.81800000e+00],
       [6.04633200e+03, 2.66000000e+00],
       [1.44144512e+05, 1.94000000e+00],
       [7.02380000e+03, 2.66000000e+00],
       [2.14173890e+05, 1.79000000e+00],
       [7.66194040e+04, 2.10000000e+00],
       [4.08547400e+03, 2.85000000e+00],
       [1.97359000e+04, 2.35000000e+00]])

In [771]:
df_0A[4:8]

,account_nickname,cnpj,age,growth_overall,initial_reported_revenue,rating,gross_value,monthly_interest_rate
4,AGENCIA LEAD,31410323000187,4.982661,0.980326,19673.92,CC,5183.83,2.50
5,AGORA MARCAS,38157946000139,2.992203,-0.484142,70000.00,CC,41821.76,2.00
6,Ativum Contabilidade,29402935000186,5.601428,0.421759,41985.00,CC,5183.66,2.50
7,Auba,13611634000122,12.328466,-0.309278,1270.00,C,1594.44,3.15


In [772]:
model.score(X_test,y_test)

0.39992035668720566

K Neighbors seems to be an overestimate...

### Decision Tree Regression for Multi-Output Credit Plan

In [773]:
from sklearn.tree import DecisionTreeRegressor
model0A = DecisionTreeRegressor()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=100)

In [774]:
model0A.fit(X_train,y_train)

DecisionTreeRegressor()

In [775]:
model0A.predict(X_test)

array([[2.0849759e+05, 1.6500000e+00],
       [4.1671400e+03, 2.6500000e+00],
       [3.2185330e+04, 3.1500000e+00],
       [3.0800000e+03, 2.6500000e+00],
       [1.0400000e+03, 3.5000000e+00],
       [1.9423976e+06, 1.3500000e+00],
       [6.2204600e+03, 2.2500000e+00],
       [2.5919300e+04, 1.6000000e+00],
       [1.0423520e+04, 2.2500000e+00],
       [5.1954460e+04, 2.5000000e+00],
       [2.5918770e+04, 2.1000000e+00],
       [1.0367530e+04, 2.2500000e+00],
       [1.5729740e+04, 2.7500000e+00],
       [1.0360329e+05, 2.2500000e+00],
       [1.0360329e+05, 2.2500000e+00],
       [3.4758902e+05, 1.6000000e+00],
       [2.1051900e+03, 3.9500000e+00],
       [8.1770310e+04, 1.8000000e+00],
       [1.6306310e+04, 2.5000000e+00],
       [1.8750433e+05, 1.5000000e+00],
       [5.2277450e+04, 2.0000000e+00],
       [2.5918500e+03, 2.4000000e+00],
       [5.2119710e+04, 2.0000000e+00]])

In [776]:
df_0A[4:8]

,account_nickname,cnpj,age,growth_overall,initial_reported_revenue,rating,gross_value,monthly_interest_rate
4,AGENCIA LEAD,31410323000187,4.982661,0.980326,19673.92,CC,5183.83,2.50
5,AGORA MARCAS,38157946000139,2.992203,-0.484142,70000.00,CC,41821.76,2.00
6,Ativum Contabilidade,29402935000186,5.601428,0.421759,41985.00,CC,5183.66,2.50
7,Auba,13611634000122,12.328466,-0.309278,1270.00,C,1594.44,3.15


In [777]:
import sklearn.metrics as sm
model0A.score(X_test,y_test)

0.42230179532639056

Decision trees seem to be a weak model moving forward.